<a href="https://colab.research.google.com/github/Khushil-Modi/CMPE-297/blob/main/Assignment%205/CMPE_297_Assignment_5(a)_Continual_Learning_using_Avalanche_library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **a) Do an end2end well annotated and commented demo of continual learning using avalanche library in colab**

## **PermutedMNIST dataset fed to SimpleMLP model**

Avalanche and its dependencies can be installed using the following command. Avalanche runs on either CPU or CUDA GPU based on availability. Since Avalanche is built on top of PyTorch, a PyTorch environment has to be created.

In [1]:
# Install Avalanche and its dependencies
!pip install git+https://github.com/ContinualAI/avalanche.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-z8izskiy
  Running command git clone -q https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-z8izskiy
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 532 kB 30.6 MB/s 
     |████████████████████████████████| 427 kB 70.7 MB/s 
     |████████████████████████████████| 1.9 MB 57.1 MB/s 
     |████████████████████████████████| 529 kB 70.8 MB/s 
     |████████████████████████████████| 168 kB 71.8 MB/s 
     |████████████████████████████████| 182 kB 75.7 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 166 kB 71.9 MB/s 
     |████████████████████████████████| 166 kB 67.1 MB/s 
     |████████████████████████████████| 162 kB 76.1 MB/s 
     |█

###Importing necessary libraries and modules.

In [2]:
import torch
from torch.nn import CrossEntropyLoss       # use CrossEntropyLoss
from torch.optim import SGD                 # use stochastic GD optimizer
from avalanche.benchmarks.classic import PermutedMNIST         # import the PermutedMNIST dataset
from avalanche.models import SimpleMLP      # import the SimpleMLP dataset
from avalanche.training.supervised import Naive                # we will use Naive training strategy

Configure the device settings by checking for availability of CUDA GPU. Instantiate a simple multi-layer perceptron model.

In [3]:
# Configure the device settings
# check for CUDA GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model instantiation
model = SimpleMLP(num_classes = 10)

Prepare the benchmark dataset. Split the dataset into train and test sets.

In [4]:
# Benchmark instantiation
perm_mnist = PermutedMNIST(n_experiences=3)
# split stream into train and test set
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



Instantiate optimizer and loss functions.

In [5]:
# define optimizer and loss function for training
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

Using Naive strategy

In [6]:
# Instantiate Continual learning strategy
strategy = Naive(
    model, optimizer, criterion, train_mb_size=32, train_epochs=2,
    eval_mb_size=32, device=device)

Perform training, evaluate the model and log results.

In [7]:
results = []
# train and test epochs
for train_task in train_stream:
    # training
    strategy.train(train_task, num_workers=4)
    # evaluate the model and store the results
    results.append(strategy.eval(test_stream))

-- >> Start of training phase << --
0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 1875/1875 [01:05<00:00, 28.71it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4924
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8593
100%|██████████| 1875/1875 [01:10<00:00, 26.56it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2637
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9238
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 313/313 [00:05<00:00, 53.77it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.1860
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9467
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 313/313 [00:05<00:00, 55.18it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp001 = 2.5854
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.0882
-- S

In [8]:
results

[{'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9237666666666666,
  'Loss_Epoch/train_phase/train_stream/Task000': 0.26372187160352867,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9467,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.18601460604071618,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001': 0.0882,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp001': 2.5854198276519775,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002': 0.112,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp002': 2.5185113651275635,
  'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.3823,
  'Loss_Stream/eval_phase/test_stream/Task000': 1.763315266273419},
 {'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9343,
  'Loss_Epoch/train_phase/train_stream/Task000': 0.22500290407339732,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9363,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.21819261346310379,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001

#**Reference:** 
#####https://analyticsindiamag.com/avalanche-a-python-library-for-continual-learning/